# Aspect harmonization

Harmonizing the aspects is the final step of Namkha calculation algorithm when we know all aspect elements and it's time to harmonize their relations using element sequences.
It is the same for all types of single Namkha (Namkhai Norbu 2022, 25), and all calculation methods. 

Defining relationships between the Elements using a concept of the Circle of Elements illustrated in (Namkhai Norbu 1999, 34):

In [1]:
from dataclasses import dataclass
from enum import unique, Enum
from typing import Optional


@unique
class Element(Enum):
     WATER = "Water"
     WOOD = "Wood"
     FIRE = "Fire"
     EARTH = "Earth"
     METAL = "Metal"


@dataclass
class ElementItem:
    mother: Element
    son: Element


@unique
class AspectName(str, Enum):
    LIFE = "life"
    BODY = "body"
    CAPACITY = "capacity"
    FORTUNE = "fortune"
    MEWA_LIFE = "mewa_life"
    MEWA_BODY = "mewa_body"
    MEWA_CAPACITY = "mewa_capacity"
    MEWA_FORTUNE = "mewa_fortune"


@dataclass
class Aspect:
    name: AspectName
    center: Element
    harmonization_sequence: tuple[Element, ...]
    is_conflicted: Optional[bool] = None


ELEMENTS_CIRCLE = {
    Element.WATER: ElementItem(
        mother=Element.METAL, son=Element.WOOD
    ),
    Element.WOOD: ElementItem(
        mother=Element.WATER, son=Element.FIRE
    ),
    Element.FIRE: ElementItem(
        mother=Element.WOOD, son=Element.EARTH
    ),
    Element.EARTH: ElementItem(
        mother=Element.FIRE, son=Element.METAL
    ),
    Element.METAL: ElementItem(
        mother=Element.EARTH, son=Element.WATER
    ),
}

Helper functions for the Harmonization algorithm:

In [2]:
def get_son(element: Element) -> Element:
    return ELEMENTS_CIRCLE[element].son


def get_mother(element: Element) -> Element:
    return ELEMENTS_CIRCLE[element].mother


def circle_forwards(start_element: Element) -> list[Element]:
    result = [start_element]
    for _ in range(4):
        result.append(get_son(result[-1]))
    return result


def circle_backwards(start_element: Element) -> list[Element]:
    result = [start_element]
    for _ in range(4):
        result.append(get_mother(result[-1]))
    return result


def shortest_path(
    first_element: Element, second_element: Element
) -> list[Element]:
    fwd = circle_forwards(first_element)
    fwd = fwd[: fwd.index(second_element) + 1]
    rev = circle_backwards(first_element)
    rev = rev[: rev.index(second_element) + 1]
    return fwd[1:] if len(fwd) < len(rev) else rev[1:]


def get_edge_element(center_element: Element) -> Element:
    return ELEMENTS_CIRCLE[center_element].mother


def are_in_conflict(
    first_element: Element, second_element: Element
) -> bool:
    if (
        first_element in [get_son(second_element), get_mother(second_element)]
        or first_element == second_element
    ):
        return False
    return True

## Harmonization algorithm

In [3]:
def harmonize_aspects(
    life: Element,
    body: Element,
    capacity: Element,
    fortune: Element,
    mewa_life: Element,
    mewa_body: Element,
    mewa_capacity: Element,
    mewa_fortune: Element,
) -> tuple[Aspect, ...]:
    edge_element = get_edge_element(life)
    result = []

    life_seq = circle_forwards(start_element=life)
    result.append(
        Aspect(name=AspectName.LIFE, center=life, harmonization_sequence=tuple(life_seq[1:]))
    )

    for element, name, harmonize_to in zip(
        [body, capacity, fortune, mewa_life, mewa_body, mewa_capacity, mewa_fortune],
        list(AspectName)[1:],
        [life] * 4 + [mewa_life] * 3,
    ):
        is_conflicted = False

        if are_in_conflict(element, harmonize_to):
            is_conflicted = True
            stripes = circle_backwards(element)
            if stripes[-1] == edge_element:
                stripes.extend([get_mother(edge_element), edge_element])
            else:
                stripes.extend(
                    shortest_path(
                        first_element=stripes[-1], second_element=edge_element
                    )
                )
        else:
            stripes = circle_forwards(element)
            if stripes[-1] != edge_element:
                stripes.extend(
                    shortest_path(
                        first_element=stripes[-1], second_element=edge_element
                    )
                )

        result.append(
            Aspect(
                name=name,
                center=stripes.pop(0),
                harmonization_sequence=tuple(stripes),
                is_conflicted=is_conflicted,
            )
        )

    return tuple(result)

## Example

In [5]:
def aspects_to_text(aspects: tuple[Aspect, ...]) -> str:
    for aspect in aspects:
        print (
            "{0}<{1}> － {2}".format(
                (aspect.name.name + ("[c]" if aspect.is_conflicted else "")).ljust(18),
                aspect.center.value.ljust(5),
                ' － '.join([element.value.ljust(5) for element in aspect.harmonization_sequence])
            )
        )

aspects_to_text(
    harmonize_aspects(
        life=Element.WOOD,
        body=Element.WATER,
        capacity=Element.WOOD,
        fortune=Element.FIRE,
        mewa_life=Element.WATER,
        mewa_body=Element.METAL,
        mewa_capacity=Element.FIRE,
        mewa_fortune=Element.METAL,
    )
)


LIFE              <Wood > － Fire  － Earth － Metal － Water
BODY              <Water> － Wood  － Fire  － Earth － Metal － Water
CAPACITY          <Wood > － Fire  － Earth － Metal － Water
FORTUNE           <Fire > － Earth － Metal － Water － Wood  － Water
MEWA_LIFE         <Water> － Wood  － Fire  － Earth － Metal － Water
MEWA_BODY         <Metal> － Water － Wood  － Fire  － Earth － Metal － Water
MEWA_CAPACITY[c]  <Fire > － Wood  － Water － Metal － Earth － Metal － Water
MEWA_FORTUNE      <Metal> － Water － Wood  － Fire  － Earth － Metal － Water
